In [1]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,297 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,783 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/r

In [9]:
import threading
import subprocess
import time
import ollama

def run_ollama_serve():
    # Usiamo Popen per non bloccare
    subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Avvia il server in un thread
thread = threading.Thread(target=run_ollama_serve)
thread.daemon = True # Il thread si chiuderà quando lo script principale termina
thread.start()

# --- Health Check Robusto ---
server_ready = False
max_wait_time = 30 # Attendi al massimo 30 secondi
start_time = time.time()
print("⏳ In attesa che il server Ollama si avvii...")

while not server_ready and time.time() - start_time < max_wait_time:
    try:
        # Prova a fare una chiamata leggera, come listare i modelli
        ollama.list()
        server_ready = True
        print("✅ Server Ollama avviato e pronto.")
    except Exception as e:
        # Il server non è ancora pronto, attendi un secondo e riprova
        time.sleep(1)

if not server_ready:
    print("❌ Impossibile avviare il server Ollama entro il tempo massimo.")
    # Qui potresti decidere di terminare lo script
    exit()



⏳ In attesa che il server Ollama si avvii...
✅ Server Ollama avviato e pronto.


In [3]:
!ollama pull llama3.1:8b
!ollama pull zephyr
!ollama pull deepseek-r1:7b

In [10]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import numpy as np
import json
import ollama
from langchain_core.prompts import PromptTemplate
from tqdm import tqdm
import time
import csv
from nomic import embed
from sklearn.metrics.pairwise import cosine_similarity

# --- PARAMETERS ---
DATA_PATH = "YOUR DRIVE PATH" # RecGym.csv
OUTPUT_PATH = "YOUR DRIVE PATH" # results_recgym.json
CSV_LOG_PATH = "YOUR DRIVE PATH" # results_log.csv

ROWS_OPTIONS = [300,600,900] 
MAX_WINDOWS = 20
EXAMPLES_PER_EX = 1
POSITIONS = ['wrist', 'pocket', 'leg']
MODELS = ["llama3.1:8b", "zephyr:latest"]

# --- HELPERS ---
def compute_stats(df_window: pd.DataFrame) -> dict:
    acc_mag = np.sqrt(df_window["A_x"]**2 + df_window["A_y"]**2 + df_window["A_z"]**2)
    stats = {
        "acc_mean_mag": acc_mag.mean(),
        "acc_std_mag": acc_mag.std(),
        "gyro_energy": df_window[["G_x","G_y","G_z"]].var().sum()
    }
    if "C_1" in df_window.columns:
        stats["cap_mean"] = df_window["C_1"].mean()
    return stats

def stats_to_text(pos_stats: dict) -> str:
    lines = []
    for pos, stats in pos_stats.items():
        lines.append(f"{pos.capitalize()}:")
        lines += [f"{k}: {v:.3f}" for k,v in stats.items()]
        lines.append("")
    return "\n".join(lines)

def reduce_dataset(df: pd.DataFrame, rows_per_pos: int) -> dict:
    windows_per_exercise = {}
    for ex in df['Workout'].unique():
        pos_data = {pos: df[(df['Workout']==ex) & (df['Position']==pos)].reset_index(drop=True)
                    for pos in POSITIONS}
        if not all(len(data) > 0 for data in pos_data.values()):
            continue

        total_windows = min(len(pos_data[pos]) for pos in POSITIONS) // rows_per_pos
        for i in range(min(total_windows, MAX_WINDOWS)):
            window = pd.concat([pos_data[pos].iloc[i*rows_per_pos:(i+1)*rows_per_pos]
                                for pos in POSITIONS], ignore_index=True)
            windows_per_exercise.setdefault(ex, []).append(window)
    return windows_per_exercise

def parse_llm_output(output: str):
    predicted, description = None, ""
    for line in output.splitlines():
        line_lower = line.lower().strip()
        if line_lower.startswith("predicted label:"):
            predicted = line.split(":",1)[1].strip()
        elif line_lower.startswith("description:"):
            description = line.split(":",1)[1].strip()
    return predicted, description

def create_few_shot_context(windows_per_exercise: dict) -> str:
    examples = []
    for ex, ex_windows in windows_per_exercise.items():
        if not ex_windows: continue
        pos_stats = {pos: compute_stats(ex_windows[0][ex_windows[0]['Position']==pos]) for pos in POSITIONS}
        stats_text = stats_to_text(pos_stats)
        examples.append(f"Movement statistics:\n{stats_text}\npredicted label: {ex}\ndescription: An example of {ex}.\n---")
    return "\n".join(examples)

def run_llm_prediction(windows_per_exercise: dict, prompt_template: str, model_name: str, few_shot_examples: str = None) -> tuple:
    results = []
    exercises = list(windows_per_exercise.keys())

    input_vars = ["stats", "labels_list"]
    if few_shot_examples is not None:
        input_vars.append("examples")

    prompt_obj = PromptTemplate(
        template=prompt_template,
        input_variables=input_vars
    )

    for ex, ex_windows in tqdm(windows_per_exercise.items(), desc=f"LLM Prediction ({model_name})"):
        for window in ex_windows[EXAMPLES_PER_EX:]:
            pos_stats = {pos: compute_stats(window[window['Position']==pos]) for pos in POSITIONS}
            stats_text = stats_to_text(pos_stats)

            prompt_args = {"stats": stats_text, "labels_list": ", ".join(exercises)}
            if few_shot_examples is not None:
                prompt_args["examples"] = few_shot_examples

            prompt_text = prompt_obj.format(**prompt_args)

            try:
                response = ollama.chat(model=model_name, messages=[{"role":"user","content":prompt_text}])
                output = response.get("message", {}).get("content","").strip()
            except Exception as e:
                output = f"ERROR: {e}"

            predicted, description = parse_llm_output(output)
            results.append({"true_label": ex, "predicted_label": predicted, "description": description})

    total_windows = len(results)
    return results, total_windows

def calculate_accuracy(results: list) -> float:
    if not results: return 0
    correct = sum(1 for r in results if r['true_label'].lower() == (r['predicted_label'] or "").lower().strip())
    return correct / len(results)

# --- ZERO-SHOT ---
def zero_shot(windows_per_exercise: dict, model_name: str) -> tuple:
    prompt_template = """
You are an AI expert in human gym exercise recognition. Data are collected using wrist, pocket, and leg sensors.

For each position, statistics shared are:
- acc_mean_mag: mean acceleration magnitude
- acc_std_mag: standard deviation of acceleration magnitude
- gyro_energy: rotational energy from gyroscope
- cap_mean: mean capacitive reading

Exercise statistics:
{stats}

Instructions:
- Analyze statistics carefully
- Determine which exercise was performed

ONLY respond in the following format:
predicted label: <exercise name, max 1 word>
description: <short explanation, max 25 words>
Do NOT write anything else.

Possible exercises:
{labels_list}
"""
    return run_llm_prediction(windows_per_exercise, prompt_template, model_name)

# --- FEW-SHOT ---
def few_shot(windows_per_exercise: dict, model_name: str) -> tuple:
    prompt_template = """
You are an AI expert in human gym exercise recognition. Data are collected using wrist, pocket, and leg sensors.

For each position, statistics shared are:
- acc_mean_mag: mean acceleration magnitude
- acc_std_mag: standard deviation of acceleration magnitude
- gyro_energy: rotational energy from gyroscope
- cap_mean: mean capacitive reading

Here are some examples:
{examples}

Now analyze the following window.

Movement statistics:
{stats}

Instructions:
- Analyze carefully
- Determine which exercise was performed

ONLY respond in the following format:
predicted label: <exercise name, max 1 word>
description: <short explanation, max 25 words>
Do NOT write anything else.

Possible exercises:
{labels_list}
"""
    few_shot_context = create_few_shot_context(windows_per_exercise)
    return run_llm_prediction(windows_per_exercise, prompt_template, model_name, few_shot_examples=few_shot_context)

# --- ZERO-SHOT RAG ---
def zero_shot_rag(windows_per_exercise: dict, KB: list, model_name: str) -> tuple:
    kb_profiles = [doc['sensor_profile'] for doc in KB]
    kb_embeddings = embed.text(texts=kb_profiles, model="nomic-embed-text-v1.5", task_type="search_document")["embeddings"]

    results = []
    exercises = list(windows_per_exercise.keys())

    prompt_template = """
You are an AI expert in human gym exercise recognition. Data are collected using wrist, pocket, and leg sensors.

For each position, statistics shared are:
- acc_mean_mag: mean acceleration magnitude
- acc_std_mag: standard deviation of acceleration magnitude
- gyro_energy: rotational energy from gyroscope
- cap_mean: mean capacitive reading

Additional knowledge base context:
{rag_context}

Now analyze the following window.

Movement statistics:
{stats}

Instructions:
- Analyze carefully
- Determine which exercise was performed

ONLY respond in the following format:
predicted label: <exercise name, max 1 word>
description: <short explanation, max 25 words>
Do NOT write anything else.

Possible exercises:
{labels_list}
"""
    prompt_obj = PromptTemplate(template=prompt_template, input_variables=["stats","labels_list","rag_context"])

    for ex, ex_windows in tqdm(windows_per_exercise.items(), desc=f"Zero-shot RAG ({model_name})"):
        for window in ex_windows[EXAMPLES_PER_EX:]:
            pos_stats = {pos: compute_stats(window[window['Position']==pos]) for pos in POSITIONS}
            stats_text = stats_to_text(pos_stats)

            query_vec = embed.text(texts=[stats_text], model="nomic-embed-text-v1.5", task_type="search_query")["embeddings"][0]
            sims = cosine_similarity([query_vec], kb_embeddings)[0]
            top_indices = np.argsort(sims)[::-1][:3]

            rag_context = "\n\n".join([
                f"- Exercise: {KB[i]['id']}\nDescription: {KB[i]['text']}\nSensor profile: {KB[i]['sensor_profile']}"
                for i in top_indices
            ])

            prompt_text = prompt_obj.format(stats=stats_text, labels_list=", ".join(exercises), rag_context=rag_context)

            try:
                response = ollama.chat(model=model_name, messages=[{"role":"user","content":prompt_text}])
                output = response.get("message", {}).get("content","").strip()
            except Exception as e:
                output = f"ERROR: {e}"

            predicted, description = parse_llm_output(output)
            results.append({"true_label": ex, "predicted_label": predicted, "description": description})

    total_windows = len(results)
    return results, total_windows

# --- EXECUTION ---
if __name__ == "__main__":
    df = pd.read_csv(DATA_PATH)
    df = df[(df['Workout'] != 'Null') & (df['Position'].isin(POSITIONS))].reset_index(drop=True)

    with open(KB_PATH, "r", encoding="utf-8") as f:
        KB = json.load(f)

    log_data = []
    json_results = {}

    for rows_per_pos in ROWS_OPTIONS:
        print(f"\n--- Rows per position: {rows_per_pos} ---")
        windows_per_exercise = reduce_dataset(df, rows_per_pos)

        for model in MODELS:
            print(f"\n🚀 Running experiments with model: {model}")

            # Zero-shot
            start_time = time.time()
            results_zero, windows_zero = zero_shot(windows_per_exercise, model)
            duration_zero = time.time() - start_time
            accuracy_zero = calculate_accuracy(results_zero)
            log_data.append([model, "zero", rows_per_pos, duration_zero, accuracy_zero, windows_zero])
            json_results[f"{model}_zero_shot_rows{rows_per_pos}"] = results_zero

            # Few-shot
            start_time = time.time()
            results_few, windows_few = few_shot(windows_per_exercise, model)
            duration_few = time.time() - start_time
            accuracy_few = calculate_accuracy(results_few)
            log_data.append([model, "few", rows_per_pos, duration_few, accuracy_few, windows_few])
            json_results[f"{model}_few_shot_rows{rows_per_pos}"] = results_few

            # Zero-shot RAG
            start_time = time.time()
            results_rag, windows_rag = zero_shot_rag(windows_per_exercise, KB, model)
            duration_rag = time.time() - start_time
            accuracy_rag = calculate_accuracy(results_rag)
            log_data.append([model, "zero_rag", rows_per_pos, duration_rag, accuracy_rag, windows_rag])
            json_results[f"{model}_zero_shot_rag_rows{rows_per_pos}"] = results_rag

            print(f"🎯 {model} | Rows per pos: {rows_per_pos} | Zero-shot Accuracy: {accuracy_zero*100:.2f}% | Time: {duration_zero:.2f}s")
            print(f"🎯 {model} | Rows per pos: {rows_per_pos} | Few-shot Accuracy: {accuracy_few*100:.2f}% | Time: {duration_few:.2f}s")
            print(f"🎯 {model} | Rows per pos: {rows_per_pos} | Zero-shot RAG Accuracy: {accuracy_rag*100:.2f}% | Time: {duration_rag:.2f}s")

    # Save JSON results
    with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
        json.dump(json_results, f, indent=4)

    # Save CSV log
    with open(CSV_LOG_PATH, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["model", "technique", "rows_per_pos", "execution_time_sec", "accuracy", "num_windows"])
        writer.writerows(log_data)

    print(f"\n✅ Results saved in {OUTPUT_PATH} and log in {CSV_LOG_PATH}")




--- Rows per position: 300 ---

🚀 Running experiments with model: llama3.1:8b


Zero-shot RAG (llama3.1:8b): 100%|██████████| 11/11 [01:56<00:00, 10.58s/it]


🎯 llama3.1:8b | Rows per pos: 300 | Zero-shot Accuracy: 6.22% | Time: 93.29s
🎯 llama3.1:8b | Rows per pos: 300 | Few-shot Accuracy: 23.92% | Time: 86.80s
🎯 llama3.1:8b | Rows per pos: 300 | Zero-shot RAG Accuracy: 10.53% | Time: 116.87s

🚀 Running experiments with model: zephyr:latest


Zero-shot RAG (zephyr:latest): 100%|██████████| 11/11 [02:11<00:00, 11.97s/it]


🎯 zephyr:latest | Rows per pos: 300 | Zero-shot Accuracy: 7.66% | Time: 193.87s
🎯 zephyr:latest | Rows per pos: 300 | Few-shot Accuracy: 11.48% | Time: 185.43s
🎯 zephyr:latest | Rows per pos: 300 | Zero-shot RAG Accuracy: 9.09% | Time: 132.21s

--- Rows per position: 600 ---

🚀 Running experiments with model: llama3.1:8b


...

🎯 zephyr:latest | Rows per pos: 600 | Zero-shot Accuracy: 4.78% | Time: 158.66s
🎯 zephyr:latest | Rows per pos: 600 | Few-shot Accuracy: 10.05% | Time: 170.97s
🎯 zephyr:latest | Rows per pos: 600 | Zero-shot RAG Accuracy: 9.09% | Time: 127.68s

--- Rows per position: 900 ---

🚀 Running experiments with model: llama3.1:8b


Zero-shot RAG (llama3.1:8b): 100%|██████████| 11/11 [02:04<00:00, 11.33s/it]


🎯 llama3.1:8b | Rows per pos: 900 | Zero-shot Accuracy: 5.31% | Time: 92.25s
🎯 llama3.1:8b | Rows per pos: 900 | Few-shot Accuracy: 17.39% | Time: 86.62s
🎯 llama3.1:8b | Rows per pos: 900 | Zero-shot RAG Accuracy: 12.56% | Time: 125.12s

🚀 Running experiments with model: zephyr:latest


Zero-shot RAG (zephyr:latest): 100%|██████████| 11/11 [02:16<00:00, 12.43s/it]

🎯 zephyr:latest | Rows per pos: 900 | Zero-shot Accuracy: 7.25% | Time: 158.87s
🎯 zephyr:latest | Rows per pos: 900 | Few-shot Accuracy: 12.08% | Time: 195.94s
🎯 zephyr:latest | Rows per pos: 900 | Zero-shot RAG Accuracy: 9.18% | Time: 137.30s

✅ Results saved in /content/drive/MyDrive/DT_SensorData/DominioGym/results_recgym.json and log in /content/drive/MyDrive/DT_SensorData/DominioGym/results_log.csv


In [ ]:
import json
import pandas as pd
from google.colab import files

# Mappatura esercizio → macro categoria (case insensitive)
CATEGORY_MAP = {
    "adductor": "lower body",
    "legcurl": "lower body",
    "legpress": "lower body",
    "squat": "lower body",

    "armcurl": "upper body",
    "benchpress": "upper body",

    "riding": "cardio",
    "ropeskipping": "cardio",
    "running": "cardio",
    "walking": "cardio",
    "stairsclimber": "cardio",
}

def evaluate_macro_accuracy(results: list[dict]):
    """Calcola accuracy a livello di macro-categoria, case insensitive"""
    correct = 0
    total = 0
    mismatches = []

    for r in results:
        true_label = r["true_label"].lower()
        pred_label = (r["predicted_label"] or "").lower()

        true_macro = CATEGORY_MAP.get(true_label, "other")
        pred_macro = CATEGORY_MAP.get(pred_label, "other")

        if true_macro == pred_macro:
            correct += 1
        else:
            mismatches.append({
                "true_label": r["true_label"],
                "predicted_label": r["predicted_label"],
                "true_macro": true_macro,
                "pred_macro": pred_macro,
                "description": r.get("description", "")
            })
        total += 1

    accuracy = correct / total if total > 0 else 0
    return accuracy, mismatches


print("Carica il file results.json...")
uploaded = files.upload()  # scegli "results.json" dal tuo PC

filename = list(uploaded.keys())[0]

with open(filename, "r", encoding="utf-8") as f:
    data = json.load(f)

results_list = []

for model, results in data.items():
    acc, mismatches = evaluate_macro_accuracy(results)
    print(f"\nModel: {model}")
    print(f"Macro Accuracy: {acc:.2f} ({len(results)} samples)")
    print("Esempi di mismatch:")
    for m in mismatches[:3]:
        print(m)

    results_list.append({
        "model": model,
        "macro_accuracy": acc,
        "n_samples": len(results)
    })

# 🔹 Step 2: Esporta in CSV
df = pd.DataFrame(results_list)
df.to_csv("macro_accuracy_results.csv", index=False)
print("\nCSV salvato: macro_accuracy_results.csv")

files.download("macro_accuracy_results.csv")

Carica il file results.json...


Saving results_recgym (3).json to results_recgym (3) (1).json

Model: llama3.1:8b_zero_shot_rows300
Macro Accuracy: 0.20 (209 samples)
Esempi di mismatch:
{'true_label': 'LegCurl', 'predicted_label': 'Walking', 'true_macro': 'lower body', 'pred_macro': 'cardio', 'description': 'Low acceleration and gyro energy levels suggest a low-impact exercise like walking.'}
{'true_label': 'LegCurl', 'predicted_label': 'Walking', 'true_macro': 'lower body', 'pred_macro': 'cardio', 'description': 'Low acceleration and rotational energy, consistent capacitive readings across sensors, suggesting a low-impact activity like walking.'}
{'true_label': 'LegCurl', 'predicted_label': 'Walking', 'true_macro': 'lower body', 'pred_macro': 'cardio', 'description': 'Low acceleration and energy values suggest a low-impact exercise with minimal rotational movement.'}

Model: llama3.1:8b_few_shot_rows300
Macro Accuracy: 0.67 (209 samples)
Esempi di mismatch:
{'true_label': 'Squat', 'predicted_label': 'Running', 'tru

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>